In [1]:
import HTSeq
HTSeq.__version__

'0.8.0'

In [2]:
import pickle

In [2]:
sam_fpath='/ifs/home/yy1533/Lab/cel-seq-pipe/demo/celseq2/smallsam/BC-15-TCTGAG.sam'
gff_fpath='/ifs/data/yanailab/refs/danio_rerio/danRer10_87/gtf/Danio_rerio.GRCz10.87.gtf.gz'

In [3]:
htseq_fpath='/ifs/home/yy1533/Lab/cel-seq-pipe/demo/celseq2/sam2umicnt/BC-15-TCTGAG.count.bk' ## generated by default htseq-count command

# MWE for single-end sam + GFF => alignments count

<http://htseq.readthedocs.io/en/master/counting.html>

In [5]:
## Prepare GFF
fh_gff = HTSeq.GFF_Reader(gff_fpath)
features = HTSeq.GenomicArrayOfSets( "auto", stranded=True )

In [4]:
feature_atrr = 'gene_id'
feature_type = 'exon'
accept_aln_qual_min = 10
is_gapped_aligner = True ## bowtie2 is False
## Bowtie2: 1) non_gapped_aligner; 2) default report best one for multiple alignment
len_umi = 6
len_bc = 6

In [6]:
for gff in fh_gff:
    if gff.type != feature_type:
        continue
    features[gff.iv] += gff.attr[feature_atrr]

In [6]:
# %%time
# pickle.dump(features, open('test.p', 'wb'))

# %%time
# features_bk = pickle.load(open('test.p', 'rb'))

In [8]:
## sam per cell
from collections import Counter

In [9]:
counts = Counter()
fh_aln = HTSeq.SAM_Reader(sam_fpath)
for aln in fh_aln:
    if not aln.aligned:
        counts["_unmapped"] += 1
        continue
    if aln.aQual < accept_aln_qual_min:
        counts["_low_qual"] += 1
        continue
#     try: # bowtie2 report best one randomly by default
#         if aln.optional_field( "NH" ) > 1:
#             counts['_multimapped'] += 1
#             continue
#     except KeyError:
#         pass
    
    gene_ids = set()
    if is_gapped_aligner:
        for aln_part in aln.cigar:
            if aln_part.type != 'M':
                continue
            for _, gene_id in features[aln_part.ref_iv].steps():
                gene_ids |= gene_id
    else: # bowtie2 is non-gapped-aligner
        for _, gene_id in features[aln.iv].steps():
            gene_ids |= gene_id

    ## union model        
    if len(gene_ids) == 1:
        gene_id = list(gene_ids)[0]
        counts[gene_id] += 1
    elif len(gene_ids) == 0:
        counts["_no_feature"] += 1
    else:
        counts["_ambiguous"] += 1

In [10]:
i = 0
for gene_id in counts:
    print(gene_id, counts[gene_id])
    i += 1
    if i == 10: break

ENSDARG00000041450 1
_unmapped 1562
ENSDARG00000014690 115
_low_qual 2023
ENSDARG00000080337 339
ENSDARG00000036162 7
ENSDARG00000018334 4
_no_feature 2100
ENSDARG00000103791 3
ENSDARG00000032430 1


In [11]:
! grep ENSDARG00000080337 {htseq_fpath}
! grep ENSDARG00000014690 {htseq_fpath}

ENSDARG00000080337	339
ENSDARG00000014690	115


## MWE for sam+GFF => UMI count 

In [13]:
fh_aln = HTSeq.SAM_Reader(sam_fpath)

aln = next(iter(fh_aln))

aln.read.name

aln.read.seq

In [12]:
from collections import defaultdict

In [13]:
def _umi_seq(name, length=6):
    ## BC-TCTGAG_UMI-CGTTAC => 
    try:
        out = name.split('_')[1][4:4+length]
    except Exception as e:
        raise(e)
    return(out)
foox='BC-TCTGAG_UMI-CGTTAC'
foo=_umi_seq(foox, 6)
print(foo)
foox='BC-TCTGAG_UMI-AAAAAA'
print(foo)

CGTTAC
CGTTAC


In [19]:
umi_cnt = defaultdict(set)
aln_cnt = Counter()
fh_aln = HTSeq.SAM_Reader(sam_fpath)
i = 0
for aln in fh_aln:
    i += 1
    if not aln.aligned:
        aln_cnt["_unmapped"] += 1
        continue
    if aln.aQual < accept_aln_qual_min:
        aln_cnt["_low_qual"] += 1
        continue
    try: # bowtie2 report best one randomly by default
        if aln.optional_field( "NH" ) > 1:
            aln_cnt['_multimapped'] += 1
            continue
    except KeyError:
        pass
    
    gene_ids = set()
    if is_gapped_aligner:
        for aln_part in aln.cigar:
            if aln_part.type != 'M':
                continue
            for _, gene_id in features[aln_part.ref_iv].steps():
                gene_ids |= gene_id
    else: # bowtie2 is non-gapped-aligner
        for _, gene_id in features[aln.iv].steps():
            gene_ids |= gene_id
    ## union model        
    if len(gene_ids) == 1:
        gene_id = list(gene_ids)[0]
        aln_cnt[gene_id] += 1
        umi_seq = _umi_seq(aln.read.name)
        umi_cnt[gene_id].add(umi_seq)
    elif len(gene_ids) == 0:
        aln_cnt["_no_feature"] += 1
    else:
        aln_cnt["_ambiguous"] += 1        

In [20]:
umi_count = Counter({x : len(umi_cnt.get(x, set())) for x in umi_cnt})

In [21]:
i = 0
for gene_id in umi_count:
    print(gene_id, umi_count[gene_id])
    i += 1
    if i == 10: break

ENSDARG00000041450 1
ENSDARG00000014690 107
ENSDARG00000080337 285
ENSDARG00000036162 7
ENSDARG00000018334 4
ENSDARG00000103791 3
ENSDARG00000032430 1
ENSDARG00000051888 8
ENSDARG00000051975 1
ENSDARG00000024540 9


In [24]:
! grep ENSDARG00000014690 {htseq_fpath}

ENSDARG00000014690	115


## Test modules



In [14]:
from celseq2.prepare_annotation_model import cook_anno_model
features2 = cook_anno_model(gff_fpath, verbose=True)

[ Wed Jun 28 11:47:02 2017 ] Processing 0 lines of GFF...
[ Wed Jun 28 11:47:41 2017 ] Processing 100,000 lines of GFF...
[ Wed Jun 28 11:48:20 2017 ] Processing 200,000 lines of GFF...
[ Wed Jun 28 11:49:00 2017 ] Processing 300,000 lines of GFF...
[ Wed Jun 28 11:49:39 2017 ] Processing 400,000 lines of GFF...


In [1]:
import pickle
from celseq2.count_umi import count_umi
features2 = pickle.load(open('./test.p', 'rb'))
umi_count2 = count_umi('/ifs/home/yy1533/Lab/cel-seq-pipe/demo/celseq2/smallsam/BC-15-TCTGAG.sam',
                       features2, 6, 10)

In [2]:
umi_count2['ENSDARG00000014690']

107